In [1]:
# import ee
# import geemap
from Py6S import *
import datetime
import math
import os
from osgeo import gdal
import rasterio as rio
# sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import rioxarray as rxr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
# from atmospheric import Atmospheric


In [2]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [81]:
startDate = ee.Date('2018-04-01')
endDate = ee.Date('2018-05-01')
# geom = ee.Geometry.Point(88.91220182000018,22.700863529688942)# Ninh Thuan region
# region = geom.buffer(60000).bounds().getInfo()['coordinates']

In [82]:
Map = geemap.Map()
jharkhand= ee.FeatureCollection('projects/ee-tamsheelansari1/assets/jharkhand')
L8_col= ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(jharkhand).filterDate(startDate,endDate).filterMetadata('CLOUD_COVER','less_than',20)
jh= L8_col.first()

Map.addLayer(jharkhand,{},'jharkhand')
Map.addLayer(jh,{'band':['B4','B3','B2'],'min':0,'max':0.25},'jh')
Map.centerObject(jharkhand,6)


Map

Map(center=[23.653915918218225, 85.5539359645643], controls=(WidgetControl(options=['position', 'transparent_b…

In [107]:
print(L8_col)
print(L8_col.size().getInfo())

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.load",
                      "arguments": {
                        "id": {
                          "constantValue": "LANDSAT/LC08/C02/T1_TOA"
                        }
                      }
                    }
                  },
                  "filter": {
                    "functionInvocationValue": {
                      "functionName": "Filter.intersects",
                      "arguments": {
                        "leftField": {
            

In [108]:
# L8_list = L8_col.toList(L8_col.size().getInfo())
# img1 = ee.Image(L8_list.get(9))

In [109]:
toa = jh

In [110]:
# from IPython.display import display, Image

# channels = ['B4','B3','B2']

# original = Image(url=toa.select(channels).getThumbUrl({
#                 'region':jharkhand,
#                 'min':0,
#                 'max':0.25
            
#                 }))
# display(original)

In [126]:
def func1(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    # h2o = 0.3
    # o3 = 0.7
    # aot = 0.8
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry =jharkhand, scale= 100).get('elevation').getInfo()
    
    km = (alt)/(1000) # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    # s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    # s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z        # solar zenith angle
    # s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(0.3)


In [127]:
func1(toa)

In [128]:
def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_OLI_B1,
            'B2':PredefinedWavelengths.LANDSAT_OLI_B2,
            'B3':PredefinedWavelengths.LANDSAT_OLI_B3,
            'B4':PredefinedWavelengths.LANDSAT_OLI_B4,
            'B5':PredefinedWavelengths.LANDSAT_OLI_B5,
            'B6':PredefinedWavelengths.LANDSAT_OLI_B6,
            'B7':PredefinedWavelengths.LANDSAT_OLI_B7,
            'B8':PredefinedWavelengths.LANDSAT_OLI_B8,
            'B9':PredefinedWavelengths.LANDSAT_OLI_B9,
                    }
        return Wavelength(bandSelect[bandname])


In [129]:
def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_BAND = {
            'B1':ESUN_L8[0],
            'B2':ESUN_L8[1],
            'B3':ESUN_L8[2],
            'B4':ESUN_L8[3],
            'B5':ESUN_L8[4],
            'B6':ESUN_L8[5],
            'B7':ESUN_L8[6],
            'B8':ESUN_L8[7],
            'B9':ESUN_L8[8],
            }
        solar_elv = jh.getInfo()['properties']['SUN_ELEVATION']
        solar_z = ee.Number(90).subtract(solar_elv).getInfo()
        
        ee_date = ee.Date('2018-05-01')
        scene_date = datetime.datetime.utcfromtimestamp(ee_date.getInfo()['value']/1000)
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        day = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (day-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = jh.select(bandname).multiply(multiplier)
        return rad
        

In [130]:
def surface_reflectance(bandname):
        # run 6S for this waveband
        s = SixS()
        s.wavelength = spectralResponseFunction(bandname)
        s.run()
        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref


In [131]:
ca  = surface_reflectance('B1')
blue = surface_reflectance('B2')
green = surface_reflectance('B3')
red = surface_reflectance('B4')
nir = surface_reflectance('B5')
swir1 = surface_reflectance('B6')
swir2 = surface_reflectance('B7')

ref = jh.select('BQA').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
ee_date = ee.Date('2018-05-01')
scene_date = datetime.datetime.utcfromtimestamp(ee_date.getInfo()['value']/1000)
solar_elv = jh.getInfo()['properties']['SUN_ELEVATION']
dateString = scene_date.strftime("%Y-%m-%d")
ref = ref.copyProperties(jh).set({              
              'AC_date':dateString,
            #   'AC_aerosol_optical_thickness':aot,
            #   'AC_water_vapour':h2o,
              'AC_version':'py6S',
              'AC_contact':'tamsheelansari1@gmail.com',
              # 'AC_ozone':o3
            })




In [117]:
# fname = ee.String(boa.get('system:index')).getInfo()
# export = ee.batch.Export.image.toAsset({
#     image= ref,
#     description= 'L8_BOA',
#     assetId = 'projects/ee-tamsheelansari1/assets/jharkhand'+'L8_BOA',
#     region = jharkhand,
#     scale = 30,
#     maxPixels = 1e13
#     })

#     # # uncomment to run the export
# export.start()
# print('exporting ' +fname + '--->done')

In [134]:
boa = ee.Image(func1(toa)) 

Map= geemap.Map()
# Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(boa,{'band':['B4','B3','B2'],'min':0,'max':0.30},'boa')
Map.centerObject(jharkhand,8)

In [135]:
Map

Map(center=[23.653915918218225, 85.5539359645643], controls=(WidgetControl(options=['position', 'transparent_b…

In [136]:
print(boa)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.mask",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "Image.constant",
          "arguments": {
            "value": {
              "constantValue": 0
            }
          }
        }
      },
      "mask": {
        "functionInvocationValue": {
          "functionName": "Image.constant",
          "arguments": {
            "value": {
              "constantValue": 0
            }
          }
        }
      }
    }
  }
})
